In [1]:
import os
from matplotlib import pyplot as plt
import torchvision.transforms as transforms
from torchvision.transforms.functional import resize
import numpy as np
from torch import nn
import torch
from unet import UNet
from loss import cross_entropy2d
from preprocessing import image_crop
from image_handle import analyze_mask_classes, analyze_mask_classes_rgb, image_color_mapping
from tqdm import tqdm
from torch.utils.data import Dataset, DataLoader
import torch
from PIL import Image
from dp_support import *
from path_handle import find_files_with_extension

In [2]:
train_image_dir = r".\image"
train_label_dir = r".\mask"

In [3]:
train_image_file_name_list,train_image_file_name_list_length = find_files_with_extension(train_image_dir,[".jpg"])
train_label_file_name_list, train_label_file_name_list_length = find_files_with_extension(train_label_dir, [".png"])
print(train_image_file_name_list,train_image_file_name_list_length)
print(train_label_file_name_list, train_label_file_name_list_length)

lenth of .\image : 6
lenth of .\mask : 6


In [4]:
torch.cuda.empty_cache()
# 检查训练数据集图像和标签文件列表长度是否一致
if train_image_file_name_list_length == train_label_file_name_list_length:
    # 继续执行
    pass
else:
    # 退出并报错
    raise ValueError("Training image file list length does not match training label file list length.")

In [5]:
val_image_dir = r".\image"
val_label_dir = r".\mask"

# 找到验证数据集图像和标签文件列表
val_image_file_name_list, val_image_file_name_list_length = find_files_with_extension(val_image_dir, [".jpg"])
val_label_file_name_list, val_label_file_name_list_length = find_files_with_extension(val_label_dir, [".png"])

lenth of .\image : 6
lenth of .\mask : 6


In [6]:
# # 检查验证数据集图像和标签文件列表长度是否一致
if val_image_file_name_list_length == val_label_file_name_list_length:
    # 继续执行
    pass
else:
    # 退出并报错
    raise ValueError("Validation image file list length does not match validation label file list length.")

In [7]:
class FCNDataset(Dataset):
    #当你使用DataLoader时，它会自动为你的数据批量添加一个批次维度（batch dimension）?

    def __init__(self, image_path_list: list, label_path_list: list, file_list_length: int):
        self.transform = transforms.Compose([
            # CustomResizeTransform(),
            transforms.ToTensor(),
            transforms.Normalize(mean=[0.485, 0.456, 0.406],
                                 std=[0.229, 0.224, 0.225]),
        ])
        # self.mask_transform = transforms.Compose([
        #     # CustomResizeTransform(),
        #     transforms.ToTensor(),
        # ])
        self.image_path_list = image_path_list[0:100]
        self.label_path_list = label_path_list[0:100]
        self.file_list_length = 100
        self.color_map = {0: 0}
        self.width = self.height = 224


    def __getitem__(self, i: int):
        # print(self.label_path_list[i])
        image = Image.open(self.image_path_list[i]).convert('RGB')
        mask = Image.open(self.label_path_list[i]).convert('L')
        
        
        image_width, image_height = image.size
        if image_width < self.width:
            if  image_height < self.height:
                image = image.resize((self.width, self.height), Image.BILINEAR)
                mask = mask.resize((self.width, self.height), Image.NEAREST)
            else:
                image = image.resize((self.width, image_height), Image.BILINEAR)
                mask = mask.resize((self.width, image_height), Image.NEAREST)
        else:
            if  image_height < self.height:
                image = image.resize((image_width, self.height), Image.BILINEAR)
                mask = mask.resize((image_width, self.height), Image.NEAREST)
            
        # image = image.resize((self.width, self.height), Image.NEAREST)
        # mask = mask.resize((self.width, self.height), Image.NEAREST)

        # print(np.array(image).shape)

        # print(analyze_mask_classes(mask))
        
        mask_np_array, self.color_map = image_color_mapping(np.array(mask), self.color_map)
        # for i in mask_np_array.tolist():
        #     print(i)
        # print(self.color_map)
        # expanded_mask = np.expand_dims(mask, axis=2)
        
      
        if self.transform != None: 

            image = self.transform(image)
            mask =  torch.from_numpy(mask_np_array)
            # mask = self.mask_transform(mask)
        # 将mask从[1, H, W]压缩为[H, W]，去掉单一的通道维度
        image, mask = image_crop(image ,mask ,height=self.height , width=self.width)
        # mask = torch.squeeze(mask, 0)
        mask = mask.squeeze(0).long()
        # for i in mask.tolist():
        #     print(i)
 
        # numpy_array = mask.numpy()

        # # 打印 NumPy 数组
        # print(numpy_array)
        # plt.imshow(mask, cmap='gray')  # 假设 mask 是灰度图像
        # plt.title('Mask')
        # plt.axis('off')
        # plt.show()

        return image, mask, self.color_map
        
    def __len__(self):
        return self.file_list_length

In [8]:
num_classes = 20
# print(color_map)
train_dataset = FCNDataset(
    train_image_file_name_list, 
    train_label_file_name_list,
    file_list_length = train_image_file_name_list_length,
)
val_dataset = FCNDataset(
    val_image_file_name_list, 
    val_label_file_name_list, 
    file_list_length = val_image_file_name_list_length,
)

In [9]:
batch_size = 1
num_workers = min([os.cpu_count(), batch_size if batch_size > 1 else 0, 8])
train_loader = DataLoader(train_dataset, 
                            batch_size=batch_size,
                            num_workers= num_workers,
                            pin_memory=True
                            )

val_loader = DataLoader(val_dataset, 
                            batch_size=batch_size,
                            num_workers= num_workers,
                            pin_memory=True
                        )


In [10]:
device = pytorch_test()
if device != None:
    # model = FCN(num_classes, step= 8).to(device)
    model = UNet(num_classes).to(device)
    lr, num_epochs = 0.001, 1
    optimizer = torch.optim.Adam(model.parameters(), lr=lr)
    train_model(model, device, train_loader, val_loader,optimizer,num_epochs, num_classes)

Using device: cuda
Using version: 2.2.1+cu121

NVIDIA GeForce RTX 3070 Laptop GPU
Total memory: 7.99951171875 GB
Allocated: 0.0 GB
Cached:    0.0 GB


NameError: name 'train_model' is not defined